In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

from scipy.optimize import minimize

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [5]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Mounted at /content/drive


In [6]:
algorithm_name = 'xgbcv'
feature_name = 'stacking2'
model_name = f'{algorithm_name}_{feature_name}_4'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [7]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [8]:
def load_data(model_names, oof_list, test_list, feature_names=None):
    for model in model_names:
        oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
        test_list.append(np.loadtxt(test_dir / f'{model}_test_pred.csv',delimiter=','))
        if feature_names != None:
            feature_names += {f'{model}_class0',f'{model}_class1',f'{model}_class2'}

In [9]:
all_oof = []
all_test = []
feature_names = []

model_names = ['xgbcv_stacking1_8',
               'lgbmcv_stacking1_5',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_2']
load_data(model_names,all_oof, all_test, feature_names)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 21),
 (80000, 21),
 ['xgbcv_stacking1_8_class2',
  'xgbcv_stacking1_8_class1',
  'xgbcv_stacking1_8_class0',
  'lgbmcv_stacking1_5_class0',
  'lgbmcv_stacking1_5_class2',
  'lgbmcv_stacking1_5_class1',
  'rfcv_stacking1_1_class2',
  'rfcv_stacking1_1_class0',
  'rfcv_stacking1_1_class1',
  'lrcv_stacking1_1_class0',
  'lrcv_stacking1_1_class1',
  'lrcv_stacking1_1_class2',
  'gbcv_stacking1_1_class1',
  'gbcv_stacking1_1_class0',
  'gbcv_stacking1_1_class2',
  'etscv_stacking1_1_class2',
  'etscv_stacking1_1_class1',
  'etscv_stacking1_1_class0',
  'adacv_stacking1_2_class2',
  'adacv_stacking1_2_class1',
  'adacv_stacking1_2_class0'])

In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [12]:
# Xgboost
stacking_params = {
    'alpha': 0.3630692785727396,
    'booster': 'gbtree',
    'colsample_bytree': 0.6606530552777278,
    'eval_metric': 'mlogloss',
    'gamma': 0.1939100165323239,
    'lambda': 0.13866050382742054,
    'learning_rate': 0.03274315715816124,
    'max_depth': 29,
    'max_leaves': 110,
    'n_estimators': 1000,
    'n_jobs': -1,
    'num_class': 3,
    'objective': 'multi:softprob',
    'predictor': 'gpu_predictor',
    'random_state': 2020,
    'subsample': 0.5,
    'tree_method': 'gpu_hist',
}

In [14]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain,'train'), (dval, 'val')]

    xgb_clf = xgb.train(stacking_params, dtrain, 5000, evals=watchlist, early_stopping_rounds=50, verbose_eval=5000)

    dtest = xgb.DMatrix(all_test)
    stk_test_pred += xgb_clf.predict(dtest) / n_splits
    stk_oof_pred[val_idx] = xgb_clf.predict(dval)
    mlogloss.append(xgb_clf.best_score)

print('mean logloss= ', np.mean(mlogloss))

[0]	train-mlogloss:1.05664	val-mlogloss:1.05692
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[308]	train-mlogloss:0.132532	val-mlogloss:0.15241

[0]	train-mlogloss:1.05669	val-mlogloss:1.05667
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[309]	train-mlogloss:0.132877	val-mlogloss:0.150634

[0]	train-mlogloss:1.05668	val-mlogloss:1.05676
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[326]	train-mlogloss:0.131939	val-mlogloss:0.150743

[0]	train-mlogloss:1.05666	val-mlogloss:1.05659
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't

# Ensemble

- cv score가 안정적이여서 Stacking 예측값와 함께 가중치 최적화함
- 가중치 최적화에는 scipy의 minimize 사용

In [15]:
def log_loss_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, oof_predictions):
        final_prediction += weight*prediction

    return log_loss(y, final_prediction)

In [16]:
oof_predictions = []
test_predictions = []

model_names = ['xgbcv_stacking1_8',
               'lgbmcv_stacking1_5',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_2']
load_data(model_names,oof_predictions, test_predictions)

oof_predictions.append(stk_oof_pred)
test_predictions.append(stk_test_pred)
len(oof_predictions), oof_predictions, len(test_predictions), test_predictions

(8, [array([[9.99987125e-01, 6.94812661e-06, 5.93832283e-06],
         [1.74016878e-03, 4.98186409e-01, 5.00073433e-01],
         [9.99749362e-01, 1.08460299e-04, 1.42140037e-04],
         ...,
         [9.99987841e-01, 6.70486952e-06, 5.49337483e-06],
         [9.99990344e-01, 4.10470875e-06, 5.49379729e-06],
         [9.99977708e-01, 1.21973208e-05, 1.01412534e-05]]),
  array([[9.99086382e-01, 2.42415838e-04, 6.71201862e-04],
         [1.09544468e-02, 4.73873586e-01, 5.15171968e-01],
         [9.96615337e-01, 6.53947760e-04, 2.73071491e-03],
         ...,
         [9.99093832e-01, 2.42417476e-04, 6.63750253e-04],
         [9.99072930e-01, 2.46704197e-04, 6.80366187e-04],
         [9.99064370e-01, 2.47872834e-04, 6.87757070e-04]]),
  array([[1.  , 0.  , 0.  ],
         [0.01, 0.53, 0.46],
         [1.  , 0.  , 0.  ],
         ...,
         [1.  , 0.  , 0.  ],
         [1.  , 0.  , 0.  ],
         [1.  , 0.  , 0.  ]]),
  array([[9.99709807e-01, 1.52406280e-04, 1.37786811e-04],
        

In [17]:
best_scores = []
weights = []

iteration = 100 # 여러 번 구한 가중치 값 중 Best score를 선택
for i in tqdm(range(iteration)):
    starting_values = np.random.uniform(size=len(oof_predictions))
    bounds = [(0,1)]*len(oof_predictions)
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})

    res = minimize(log_loss_func,
                   starting_values,
                   method = 'SLSQP',
                   bounds = bounds,
                   constraints = cons,
                   options={'maxiter': 500})
    best_scores.append(res['fun'])
    weights.append(res['x'])

weights = weights[np.argmin(best_scores)]
print('\n')
print('Ensemble score: {}'.format(np.min(best_scores)))
print('Best Weights: {}'.format(weights))

100%|██████████| 100/100 [32:48<00:00, 19.69s/it]



Ensemble score: 0.14990054651454768
Best Weights: [7.35336367e-01 6.50521303e-19 1.79273480e-02 2.16840434e-18
 1.12315844e-01 9.34782696e-02 2.55376899e-02 1.54044822e-02]


In [18]:
final_prediction = 0

# 가중치 적용
for weight, p in zip(weights, test_predictions):
    final_prediction += weight*p

print(final_prediction)

[[1.31449846e-04 1.65199554e-01 8.34668971e-01]
 [9.99938602e-01 2.33539016e-05 3.80434194e-05]
 [1.10045849e-04 8.53442845e-02 9.14545695e-01]
 ...
 [9.99952152e-01 1.61747903e-05 3.16729400e-05]
 [9.42995343e-05 5.35184620e-02 9.46387257e-01]
 [9.99949520e-01 1.74922381e-05 3.29792201e-05]]


# 제출 파일 및 기타 파일 생성

In [19]:
# # submission 파일 생성

# sub = pd.read_csv(sample_file)
# sub[target_column] = np.argmax(stk_test_pred, axis=1)
# sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(final_prediction, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [20]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [21]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')

In [22]:
0.938225

0.938